## 3.1 Deep Learning 구현 장비

### 3.1.1 장비 구성

- CPU: Core의 개수 보다 단일 클럭 높아야 함.
    - 권장: i7

- RAM: 메모리 다다익선
    - 권장: 64GB
    
- GPU: 메모리 클수록 좋음(비용 비쌈)
    - 권장: RTX 2080Ti
    
- Power Supply: 비싸고 검증된 브랜드

- Cooling: GPU 발열 심해 쿨링 시스템 중요.


### Why Pytorch?

비슷한 레벨의 구현 난도를 가정 시, Pytorch가 TF 비해 뛰어난 생산성 

장점:

- Python First, 깔끔한 코드

- Numpy와 뛰어난 호환성

- Autograd

- Dynamic graph


## 3.3 Pytorch Tutorial

### 3.3.1 Tensor

Pytorch's tensor == Numpy's ndarray (같은 개념)

파이토치 연산 수행의 가장 기본적인 객체

### 3.3.2 Autograd

**Autograd**: 자동으로 미분 및 역전파 수행 함수

Pytorch는 tensor들 간에 연산을 수행할 때마다 동적으로 연산 그래프(computation graph) 생성하여 연산 결과물이 어떤 tensor로 부터 어떤 연산을 통해 왔는지 추적

In [10]:
import torch

x = torch.FloatTensor([1,2])
y = torch.FloatTensor([1,2])
y.requires_grad_(True)  

z = (x+y) + torch.FloatTensor([1,2])

Keras, TF는 **미리 정의한 연산들을 컴파일 통해 고정 후**, **정해진 입력**에 맞춰 tensor를 feedforward 해야한다.

Pytorch는 **정해진 연산이 없고**, Model은 학습해야할 parameter tensor만 미리 알고 있음, 그 weight parameter들이 어떠한 연산을 통해 학습 또는 연산에 관여하는지는 알 수 없다. (연산 수행된 직후에 알 수 있다.)

**기울기 구할 필요 없는 연산의 경우:** **with torch.no_grad()**로 수행 가능

$\Rightarrow$ 역전파 알고리즘 수행이 필요 없는 비 학습 과정, 예측(prediction), 추론(inference) 수행시 유용 $\to$ 연산 속도, 메모리 측면 이점

In [11]:
import torch

x = torch.FloatTensor(2,2)
y = torch.FloatTensor(2,2)
y.requires_grad_(True)

with torch.no_grad():
    z = (x+y) + torch.FloatTensor(2,2)

### 3.3.3 Feedforward

선형 계층(Linear laye) or 완전연결계층(Fully-connected layer) 구현

$y = xW + b\\
where\ x \in \mathbb{R}^{M\times N}, W \in \mathbb{R}^{N\times P} and\ b\in \mathbb{R}^P.\\
Thus, y \in \mathbb{R}^{M\times P}$
- input matrix $x$: $M \times N$
- weight matrix W: $M \times P$
- bias vector b: P

위 수식에서는 $x$의 표기는 벡터지만, 딥러닝 수행 시 **미니배치(mini-batch)** 기준으로 수행, x가 2차원 행렬이라고 가정

$y = f(x;\theta)\ where\ \theta=\{W,b\}$

In [12]:
import torch

def linear(x, W, b):
    y = torch.mm(x,W) + b
    return y

In [13]:
x = torch.FloatTensor(16, 10)
W = torch.FloatTensor(10, 5)
b = torch.FloatTensor(5)

y = linear(x, W, b)

### 3.3.4 nn.Module

Pytorch **nn.Module** class 제공, 사용자 필요한 모델 구조 구현 도움

특징: 
- nn.Module 상속 객체 안에 nn.Module 상속 객체를 선언하여 변수로 사용 가능
- nn.Module의 forward() 함수를 override 하여 feedforward를 구현 가능

In [14]:
import torch
import torch.nn as nn

class MyLinear(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        
        self.W = torch.FloatTensor(input_size, output_size)
        self.b = torch.FloatTensor(output_size)
        
    def forward(self, X):
        y = torch.mm(X, self.W) + self.b
        
        return y

In [15]:
x = torch.FloatTensor(16, 10)
linear = MyLinear(10,5)
y = linear(x)

**parameters()** 함수는 모듈 내에 선언된 학습이 필요한 parameters을 변환하는 **iterator**이다. 

In [16]:
params = [p.size() for p in linear.parameters()]
print(params)

[]


학습이 필요한 파라미터가 없기 때문에 빈 리스트가 반환된다.

**신경망의 학습 파라미터는 단순한 tensor가 아니기에 파라미터로 등록되어야 한다.**

파라미터로 등록하기 위해서는 **Parameter class** 로 래핑하자.

In [17]:
class MyLinear(nn.Module):
    def __init__(self, input_size, output_size):
        super(MyLinear, self).__init__()
        
        self.W = nn.Parameter(torch.FloatTensor(input_size, output_size), requires_grad=True)
        self.b = nn.Parameter(torch.FloatTensor(output_size), requires_grad=True)
        
    def forward(self, x):
        y = torch.mm(x, self.W) + self.b
        
        return y

In [18]:
params = [p.size() for p in linear.parameters()]
print(params)

[]


In [19]:
# 정리

class MyLinear(nn.Module):
    def __init__(self, input_size, output_size):
        super(MyLinear, self).__init__()
        
        self.linear = nn.Linear(input_size, output_size)
        
    def forward(self, x):
        y = self.linear(x)
        
        return y

In [20]:
x = torch.FloatTensor(16, 10)
linear = MyLinear(10,5)
y = linear(x)

In [21]:
print(linear)

MyLinear(
  (linear): Linear(in_features=10, out_features=5, bias=True)
)


### 3.3.5 역전파 수행

**Back-Propagation Algorithm: feedforward를 통해 얻은 값을 실제 정답값과의 차이를 계산해 오류(손실)을 뒤로 전달(back-propagation)**

$x \in \mathbb{R}^{16 \times 10}, \theta=\{W, b\}\ and\ W\in \mathbb{R}^{10 \times 5}, b\in \mathbb{R}^{5} \\
\hat{y} = x\cdot W + b \\
where\ \hat{y}\in \mathbb{R}^{16\times 5}\\
\\
\mathcal{L}(\theta) = ||y - \hat{y}||_2^2 \\
\nabla_{\theta} \mathcal{L}(\theta) = \nabla_\theta ||y - \hat{y}||_2^2$

In [22]:
objective = 100

x = torch.FloatTensor(16, 10)
linear = MyLinear(10, 5)
y = linear(x)
loss = (objective - y.sum())

### 3.3.6 train() & eval()

train(), eval()을 사용하면 훈련, 추론 모드를 쉽게 전환할 수 있다.

nn.Module을 상속받아 생성된 객체 기본적으로 **훈련 모드**

eval()을 사용하여 추론 모드로 바꾸면, Dropout or Batch-Normalization 같은 학습과 추론시 다른 forward() 동작을 하는 Module들에 대해서도 각 상황에 따라 올바르게 동작한다. 

**추론(inference)가 끝나면 train()을 선언하여 다시 train mode로 돌아가게 해야한다.**


### 3.3.7 선형회귀분석 예제

1. 임의로 생성된 tensor들을
2. 근사하고자 하는 정답 함수에 넣어 정답(y) 구함
3. 그 정답과 신경망을 통과한 $\hat{y}$와의 차이(error)를 MSE(평균제곱오차)통해 구해
4. Stochastic Gradient Descent(SGD, 확률적 경사 하강법)을 통해 최적화 진행

MSE: 

$\mathcal{L}_{MSE}(\hat{y}, y) = \frac{1}{N} \sum^{N}_{i=1}(\hat{y}_i - y_i)^2$

In [32]:
import random
import torch
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(MyModel, self).__init__()
        
        self.linear = nn.Linear(input_size, output_size) 
        
    def forward(self, x):
        y = self.linear(x)
        
        return y

임의의 함수 $f$가 동작한다고 가정, 함수 $f$가 내부적으로 어떻게 동작하는지 파악하려면 **손실함수를 최소로 만드는 parameter $\theta$를 찾아 함수 $f$를 근사해야 한다.**

$y = f(x_1, x_2, x_3) = 3x_1 + x_2 -2x_3\\
\hat{y} = \hat{f}(x_1, x_2, x_3;\theta)\\
\hat{\theta} = argmax_{\theta \in \Theta} \mathcal{L}(\hat{y}, y)$

In [33]:
def ground_truth(x):
    return 3 * x[:, 0] + x[:, 1] - 2 * x[:, 2]

In [39]:
def train(model, x, y, optim):
    optim.zero_grad()
    
    y_hat = model(x)
    loss = ((y - y_hat)**2).sum() / x.size(0)
    loss.backward()
    optim.step()
    return loss.data

In [40]:
# hyperparameter
batch_size = 1
n_epochs = 1000
n_iter = 1000

model = MyModel(3, 1)
optim = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.1)

print(model)

MyModel(
  (linear): Linear(in_features=3, out_features=1, bias=True)
)


In [41]:
for epoch in range(n_epochs):
    avg_loss = 0

    for i in range(n_iter):
        x = torch.rand(batch_size, 3)
        y = ground_truth(x.data)

        loss = train(model, x, y, optim)

        avg_loss += loss
    avg_loss = avg_loss / n_iter

    # simple test sample to check the network.
    x_valid = torch.FloatTensor([[.3, .2, .1]])
    y_valid = ground_truth(x_valid.data)

    model.eval()
    y_hat = model(x_valid)
    model.train()

    print(avg_loss, y_valid.data[0], y_hat.data[0, 0])

    if avg_loss < .001: # finish the training if the loss is smaller than .001.
        break

tensor(2.0311) tensor(0.9000) tensor(0.2590)
tensor(1.2999) tensor(0.9000) tensor(0.4634)
tensor(0.9590) tensor(0.9000) tensor(0.5817)
tensor(0.8494) tensor(0.9000) tensor(0.6696)
tensor(0.7670) tensor(0.9000) tensor(0.7283)
tensor(0.6828) tensor(0.9000) tensor(0.7737)
tensor(0.6433) tensor(0.9000) tensor(0.8029)
tensor(0.6332) tensor(0.9000) tensor(0.8310)
tensor(0.6373) tensor(0.9000) tensor(0.8564)
tensor(0.5910) tensor(0.9000) tensor(0.8588)
tensor(0.5825) tensor(0.9000) tensor(0.8602)
tensor(0.5835) tensor(0.9000) tensor(0.8697)
tensor(0.5441) tensor(0.9000) tensor(0.8718)
tensor(0.5101) tensor(0.9000) tensor(0.8761)
tensor(0.5125) tensor(0.9000) tensor(0.8849)
tensor(0.4567) tensor(0.9000) tensor(0.8919)
tensor(0.4952) tensor(0.9000) tensor(0.8985)
tensor(0.4560) tensor(0.9000) tensor(0.9006)
tensor(0.4325) tensor(0.9000) tensor(0.9025)
tensor(0.4230) tensor(0.9000) tensor(0.9093)
tensor(0.3637) tensor(0.9000) tensor(0.9124)
tensor(0.3961) tensor(0.9000) tensor(0.9209)
tensor(0.3

tensor(0.0018) tensor(0.9000) tensor(0.9429)
tensor(0.0016) tensor(0.9000) tensor(0.9423)
tensor(0.0016) tensor(0.9000) tensor(0.9422)
tensor(0.0016) tensor(0.9000) tensor(0.9420)
tensor(0.0016) tensor(0.9000) tensor(0.9414)
tensor(0.0015) tensor(0.9000) tensor(0.9413)
tensor(0.0015) tensor(0.9000) tensor(0.9411)
tensor(0.0015) tensor(0.9000) tensor(0.9413)
tensor(0.0014) tensor(0.9000) tensor(0.9406)
tensor(0.0015) tensor(0.9000) tensor(0.9404)
tensor(0.0014) tensor(0.9000) tensor(0.9400)
tensor(0.0013) tensor(0.9000) tensor(0.9395)
tensor(0.0012) tensor(0.9000) tensor(0.9390)
tensor(0.0013) tensor(0.9000) tensor(0.9389)
tensor(0.0012) tensor(0.9000) tensor(0.9382)
tensor(0.0012) tensor(0.9000) tensor(0.9381)
tensor(0.0011) tensor(0.9000) tensor(0.9380)
tensor(0.0011) tensor(0.9000) tensor(0.9378)
tensor(0.0011) tensor(0.9000) tensor(0.9372)
tensor(0.0010) tensor(0.9000) tensor(0.9372)
tensor(0.0010) tensor(0.9000) tensor(0.9367)


## Summary

1. nn.Module Class 상속받아 (forward 함수를 통해) 모델 아키텍처 클래스 선언

2. 해당 클래스 객체 생성

3. SGD나 Adam 등의 옵티마이저를 생성하고, 생성한 모델의 파라미터를 최적화 대상으로 등록

4. 데이터로 미니배치를 구성하여 피드포워드 연산 그래프 생성

5. 손실 함수를 통해 최종 결과값(scalar)과 손실값(loss) 계산

6. 손실에 대해서 backward() 호출 $\to$ 연산 그래프 상의 tensors의 기울기가 채워짐

7. 3번의 옵티마이저에 step()을 호출하여 gradient descent 1step 수행

8. 4번으로 돌아가 수렴 조건이 만족할 때가지 반복 수행